#### PPGIA - Unifor - Prof. Daniel - Processamento e analise de imagens digitais - Trabalho09 - Classificação de Caracteres em Japonês, Chines e Coreano

#### Importação de Bibliotecas

In [1]:
import numpy as np
import pandas as pd
import cv2 
print(cv2.__version__)
#Path Google Colab para CV2
from google.colab.patches import cv2_imshow
from google.colab import files
import dlib
from skimage import io
from PIL import Image 
%matplotlib inline
#A linha acima é necessária para mostrar gráficos Matplotlib dentro de um Notebook Jupyter
from matplotlib import pyplot as plt
#import matplotlib.pylab as plt
import sys
import ipywidgets as widgets
from IPython.display import display
import scipy
from scipy import interpolate
#import face_recognition
import pickle
import gzip

4.1.2


In [2]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.core import Dense
from tensorflow.keras.optimizers import SGD
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_datasets as tfds

#### Upload do arquivo

In [ ]:
#Upload do arquivo de imagens que será utilizado no exemplo (lena.png)
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving dataset.pickle to dataset.pickle
User uploaded file "dataset.pickle" with length 7686353 bytes


In [ ]:
with gzip.open('/content/dataset.pickle','rb') as f:
  x_train, y_train, x_test, y_test = pickle.load(f)

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]


Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [ ]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [ ]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6
469/469 [==============================] - 9s 6ms/step - loss: 0.3626 - sparse_categorical_accuracy: 0.8995 - val_loss: 0.1825 - val_sparse_categorical_accuracy: 0.9495
Epoch 2/6
469/469 [==============================] - 1s 3ms/step - loss: 0.1575 - sparse_categorical_accuracy: 0.9542 - val_loss: 0.1314 - val_sparse_categorical_accuracy: 0.9614
Epoch 3/6
469/469 [==============================] - 2s 3ms/step - loss: 0.1112 - sparse_categorical_accuracy: 0.9681 - val_loss: 0.1120 - val_sparse_categorical_accuracy: 0.9666
Epoch 4/6
469/469 [==============================] - 1s 3ms/step - loss: 0.0855 - sparse_categorical_accuracy: 0.9752 - val_loss: 0.0906 - val_sparse_categorical_accuracy: 0.9729
Epoch 5/6
469/469 [==============================] - 1s 3ms/step - loss: 0.0684 - sparse_categorical_accuracy: 0.9802 - val_loss: 0.0820 - val_sparse_categorical_accuracy: 0.9755
Epoch 6/6
469/469 [==============================] - 2s 3ms/step - loss: 0.0569 - sparse_categorical_accu

In [ ]:
model = Sequential()
model.add(Dense(64, input_shape=(64, 64), activation="sigmoid"))
model.add(Dense(64, activation="sigmoid"))
model.add(Dense(10, activation="softmax"))

In [ ]:
#Avaliar a Rede Neural
print("[INFO] avaliando a rede neural...")
predictions = model.predict(x_test, batch_size=64)
#print(classification_report(y_test, predictions))

[INFO] avaliando a rede neural...


In [ ]:
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0,100), H.history["loss"], label="train_loss")
plt.plot(np.arange(0,100), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0,100), H.history["acc"], label="train_acc")
plt.plot(np.arange(0,100), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.show()

NameError: ignored

<Figure size 432x288 with 0 Axes>